## Setup

Import SymPy for manipulating equations and quantities. First two imports are quite broad, but can be nice to have in an interactive environment. SciPy, Pandas, and Seaborn are used for numerical modeling. IPython's `display` function will be used throughout as an alternative for `print()`.

In [1]:
%load_ext autoreload
%autoreload 2

from sympy.abc import *
from sympy import *
from sympy import S
import sympy
from sympy.physics.units.quantities import Quantity
from sympy.physics.units import day

from scipy.integrate import solve_ivp
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from pathlib import Path
import tomlkit

import IPython.display
from IPython.display import display, Markdown, Image

import modeling_utils
import numpy as np
import kdl

In [2]:
def print_md(*items, sep=' '):
    display(Markdown(data=sep.join([str(item) for item in items])))

pmd = print_md

From https://stackoverflow.com/a/51138039. The expression is simplified first to help combine and eliminate units. We make an exception for values of $-1$, since SymPy might represent some division operations as something like `Pow(units, Integer(-1))`, and substituting $1$ for $-1$ completely changes the meaning of the expression in that context. The absolute value of the final expression is taken to replace things like `Mul(-1, units)` with `units`.

In [3]:
def units(expr):
    expr = simplify(expr)
    return abs(expr.subs({x: 1 for x in expr.args
                               if not x.has(Quantity)
                                  and x != -1}))

We'll be creating (and recreating) many symbols in this document, so this is a convenience function for the sake of the interactive environment.

In [4]:
def create_symbol_globals(symbols):
    global_vars = globals()
    for symbol in symbols:
        if symbol.name.isidentifier():
            global_vars[symbol.name] = symbol

Similarly, this will help us evaluate the stability of a system using the Jacobian and eigenvalues.

In [5]:
def run_jacobian_steps(jacobian, subs):
    new_jacobian = jacobian.subs(subs).subs(subs)
    sub_str = ', '.join([
        f'''${latex(sym.subs({free_sym: 0
                            for free_sym in sym.free_symbols}))
           }={latex(val)}$''' for sym, val in subs.items()])
    pmd(f'For {sub_str}, the Jacobian is')

    J = MatrixSymbol('J', jacobian.shape[0], jacobian.shape[1])
    
    #display(Eq(J, simplify(new_jacobian)))
    display(Eq(J, new_jacobian))
    
    for val, mult in new_jacobian.eigenvals().items():
        #val = simplify(val)
        output = f'${latex(val)}$ is an eigenvalue' \
                 f' with a multiplicity of {mult} '
        pmd(output)
        print()

And this will allow us to construct numerical models from the symbolic equations and parameters.

In [6]:
def mk_model(system, unknowns, params, drugs):

    model = Matrix([system[var].rhs
                    for var in unknowns
                    if var in system]
                  ).subs(params)
    
    display(Eq(Matrix([system[var].lhs
                       for var in unknowns
                       if var in system]), model))

    #drug_level_diffs = [total.derivative() for total in drugs]
    drug_level_diffs = [lambda _: 0 for total in drugs]
    
    lambdafied = lambdify(unknowns, model)
    
    def eval_model(t, y):
        return list(lambdafied(t, *y[:-2], *[levels(t) for levels in drugs])[:,0]) + [level(t) for level in drug_level_diffs]
    return eval_model

While SymPy already has a `day` unit, we need to create quantities for cells and the dose of the CTL stimulant.

In [7]:
cell = Quantity('cell')
dose = Quantity('dose')

Create representations of the functions $N$, $L$, $T$, and their derivatives. We keep $t$ unitless here, since `sympy.diff()` can't handle things like `t*day`. The derivatives are basically only for cosmetic purposes.

In [8]:
t = symbols('t')

N = Function('N')(t) * cell
Np = N.diff(t)/day

L = Function('L')(t) * cell
Lp = L.diff(t)/day

T = Function('T')(t) * cell
Tp = T.diff(t)/day

I_2 = Function('I_2')(t) * dose
I_2p = I_2.diff(t)/day

C = Function('C')(t) * dose
Cp = C.diff(t)/day

Create symbols for the model parameters and assign units to them to double-check the results we'll have from the dimensional analysis and non-dimensionalization.

In [9]:
base_params = symbols('''a_1, b, a_2, alpha_1, k_i
                    r_1, mu, beta_1,
                    c, d, alpha_2, beta_2, k_c,
                    h_i, h_c''',
                 positive=True, real=True)
create_symbol_globals(base_params)

a_1 = a_1/day
b = b/cell
a_2 = a_2/day
c = c/day
d = d/cell
r_1 = r_1/cell/day
mu = mu/day
alpha_1 = alpha_1/cell/day
alpha_2 = alpha_2/cell/day
beta_1 = beta_1/cell/day
beta_2 = beta_2/cell/day
k_i = k_i*cell/dose/day
k_c = k_c*cell/dose/day
h_i = h_i*day
h_c = h_c*day

t = t*day

## Dimensional Analysis

### Basic Consistency check
First we construct the system of equations and confirm the units are consistent.

In [10]:
system = {
    N: Eq(Np, a_1*N*(1-b*N) - a_2*N - alpha_1*N*T + k_i*I_2),
    L: Eq(Lp, r_1*N*T - mu*L - beta_1*L*T),
    T: Eq(Tp, c*T*(1-d*T) - alpha_2*N*T - beta_2*L*T - k_c*C),
    I_2: Eq(I_2p, -ln(2)*I_2.subs({t/day: 0})*2**(-t/h_i)/h_i),
    C: Eq(Cp, -ln(2)*C.subs({t/day: 0})*2**(-t/h_c)/h_c),
}

for equation in system.values():
    display(equation)
    lhs_units = units(equation.lhs)
    rhs_units = units(equation.rhs)
    if lhs_units == rhs_units:
        print(f'Units are consistent ({lhs_units})')
    else:
        print('Inconsistent units!')
        print('LHS: ', lhs_units)
        print('RHS: ', rhs_units)

Eq(cell*Derivative(N(t), t)/day, cell*a_1*(-b*N(t) + 1)*N(t)/day - cell*a_2*N(t)/day - cell*alpha_1*N(t)*T(t)/day + cell*k_i*I_2(t)/day)

Units are consistent (cell/day)


Eq(cell*Derivative(L(t), t)/day, -cell*beta_1*L(t)*T(t)/day - cell*mu*L(t)/day + cell*r_1*N(t)*T(t)/day)

Units are consistent (cell/day)


Eq(cell*Derivative(T(t), t)/day, -cell*alpha_2*N(t)*T(t)/day - cell*beta_2*L(t)*T(t)/day + cell*c*(-d*T(t) + 1)*T(t)/day - cell*k_c*C(t)/day)

Units are consistent (cell/day)


Eq(dose*Derivative(I_2(t), t)/day, -dose*I_2(0)*log(2)/(2**(t/h_i)*day*h_i))

Units are consistent (dose/day)


Eq(dose*Derivative(C(t), t)/day, -dose*C(0)*log(2)/(2**(t/h_c)*day*h_c))

Units are consistent (dose/day)


### Nondimensionalization
Now we nondimensionalize the model.

The initial equations are:

$N'(t) = \frac{d N(t)}{dt} = a N(t)(1-b N(t)) - \alpha_1 N(t) T(t)$

$L'(t) = \frac{d L(t)}{dt} = r_1 N(t) T(t) + r_2 I_0 2^{-\frac{t}{h}} - \mu L(t) - \beta_1 L(t) T(t)$

$T'(t) = \frac{d T(t)}{dt} = c T(t) (1-d T(t)) - \alpha_2 N(t) T(t) - \beta_2 L(t) T(t)$

$D'(t) = -\frac{\log(2)}{h} D_0 2^{-\frac{t}{h}}$

Which we rewrite with the following substitutions:

$\tilde{N} = \frac{\alpha_2}{\mu}N
 \left[\frac{cell^{-1} day^{-1}}{day^{-1}} cell \right]
 \Rightarrow N = \frac{\mu}{\alpha_2}\tilde{N}$

$\tilde{L} = \frac{\alpha_1 \alpha_2}{r_1 \mu} L
 \left[\frac{(cell^{-1} day^{-1}) (cell^{-1} day^{-1})}{(cell^{-1} day^{-1}) (day^{-1})} cell \right]
 \Rightarrow L = \frac{r_1 \mu}{\alpha_1 \alpha_2} \tilde{L}$

$\tilde{T} = \frac{\alpha_1}{\mu}T
 \left[\frac{cell^{-1} day^{-1}}{day^{-1}} cell \right]
 \Rightarrow T = \frac{\mu}{\alpha_1}\tilde{T}$

$\tau = \mu t
 \left[(day^{-1}) (day)\right]
 \Rightarrow t = \frac{1}{\mu} \tau
 \Rightarrow dt = \frac{1}{\mu} d \tau$

$\tilde{h} = \mu h
 \left[(day^{-1}) (day)\right]
 \Rightarrow h = \frac{1}{\mu} \tilde{h}$

$\tilde{D} = \frac{r_2}{T(0) \mu} D
 \left[\frac{day^{-1} dose^{-1} cell}{(cell)(day^{-1})} dose \right]
 \Rightarrow D = \frac{T(0) \mu}{r_2} \tilde{D}$

For equation 1, this gives us:

$dN = (a_1 N (1 - b N) - a_2 N - \alpha_1 N T)dt$

$d(\frac{\mu}{\alpha_2}\tilde{N}) = (a_1 (\frac{\mu}{\alpha_2}\tilde{N}) (1 - b (\frac{\mu}{\alpha_2}\tilde{N})) - a_2 (\frac{\mu}{\alpha_2}\tilde{N}) - \alpha_1 (\frac{\mu}{\alpha_2}\tilde{N}) (\frac{\mu}{\alpha_1}\tilde{T}))(\frac{1}{\mu} d \tau)$

$d\tilde{N} = (\frac{\alpha_2}{\mu^2})(a_1 (\frac{\mu}{\alpha_2}\tilde{N}) (1 - b (\frac{\mu}{\alpha_2}\tilde{N})) - a_2 (\frac{\mu}{\alpha_2}\tilde{N}) - \alpha_1 (\frac{\mu}{\alpha_2}\tilde{N}) (\frac{\mu}{\alpha_1}\tilde{T}))d \tau$

$d\tilde{N} = (\frac{1}{\mu^2})(a_1 (\mu\tilde{N}) (1 - b (\frac{\mu}{\alpha_2}\tilde{N})) - a_2 (\mu\tilde{N}) - \alpha_1 (\mu\tilde{N}) (\frac{\mu}{\alpha_1}\tilde{T}))d \tau$

$d\tilde{N} = (\frac{1}{\mu})(a_1 (\tilde{N}) (1 - b (\frac{\mu}{\alpha_2}\tilde{N})) - a_2 (\tilde{N}) - \alpha_1 (\tilde{N}) (\frac{\mu}{\alpha_1}\tilde{T}))d \tau$

$d\tilde{N} = (\frac{1}{\mu})(a_1 (\tilde{N}) (1 - b (\frac{\mu}{\alpha_2}\tilde{N})) - a_2 (\tilde{N}) - (\tilde{N}) (\mu\tilde{T}))d \tau$

$d\tilde{N} = (\frac{1}{\mu})(a_1 \tilde{N} (1 - \frac{b \mu}{\alpha_2}\tilde{N}) - a_2 \tilde{N} - \mu \tilde{N} \tilde{T})d \tau$

$d\tilde{N} = (\frac{a_1}{\mu} \tilde{N} (1 - \frac{b \mu}{\alpha_2}\tilde{N}) - \frac{a_2}{\mu} \tilde{N} - \tilde{N} \tilde{T})d \tau$

$\tilde{N}' = \frac{a_1}{\mu} \tilde{N} (1 - \frac{b \mu}{\alpha_2}\tilde{N}) - \frac{a_2}{\mu} \tilde{N} - \tilde{N} \tilde{T}$

For equation 2, this gives us:

$dL = (r_1 N T + r_2 D - \mu L - \beta_1 L T)dt$

$d(\frac{r_1 \mu}{\alpha_1 \alpha_2} \tilde{L}) = (r_1 (\frac{\mu}{\alpha_2}\tilde{N}) (\frac{\mu}{\alpha_1}\tilde{T}) + r_2 (\frac{T(0) \mu}{r_2} \tilde{D}) - \mu (\frac{r_1 \mu}{\alpha_1 \alpha_2} \tilde{L}) - \beta_1 (\frac{r_1 \mu}{\alpha_1 \alpha_2} \tilde{L}) (\frac{\mu}{\alpha_1}\tilde{T}))(\frac{1}{\mu} d \tau)$

$d\tilde{L} = (\frac{\alpha_1 \alpha_2}{r_1 \mu^2})((\frac{r_1 \mu^2}{\alpha_1 \alpha_2}) \tilde{N} \tilde{T} + (T(0) \mu) \tilde{D} - (\frac{r_1 \mu^2}{\alpha_1 \alpha_2} \tilde{L}) - (\frac{\beta_1 r_1 \mu^2}{\alpha_1^2 \alpha_2}) \tilde{L} \tilde{T})d \tau$

$d\tilde{L} = (\tilde{N} \tilde{T} + (\frac{T(0) \alpha_1 \alpha_2}{r_1 \mu}) \tilde{D} - \tilde{L} - (\frac{\beta_1}{\alpha_1}) \tilde{L} \tilde{T})d \tau$

$\tilde{L}' = \tilde{N} \tilde{T} + \frac{T(0) \alpha_1 \alpha_2}{r_1 \mu} \tilde{D} - \tilde{L} - \frac{\beta_1}{\alpha_1} \tilde{L} \tilde{T}$

For equation 3, this gives us:

$dT = (c T (1 - d T) - \alpha_2 N T - \beta_2 L T)dt$

$d(\frac{\mu}{\alpha_1}\tilde{T}) = (c (\frac{\mu}{\alpha_1}\tilde{T}) (1 - d (\frac{\mu}{\alpha_1}\tilde{T})) - \alpha_2 (\frac{\mu}{\alpha_2}\tilde{N}) (\frac{\mu}{\alpha_1}\tilde{T}) - \beta_2 (\frac{r_1 \mu}{\alpha_1 \alpha_2} \tilde{L}) (\frac{\mu}{\alpha_1}\tilde{T}))(\frac{1}{\mu} d \tau)$

$d\tilde{T} = (\frac{\alpha_1}{\mu^2})(c (\frac{\mu}{\alpha_1}\tilde{T}) (1 - d (\frac{\mu}{\alpha_1}\tilde{T})) - \alpha_2 (\frac{\mu}{\alpha_2}\tilde{N}) (\frac{\mu}{\alpha_1}\tilde{T}) - \beta_2 (\frac{r_1 \mu}{\alpha_1 \alpha_2} \tilde{L}) (\frac{\mu}{\alpha_1}\tilde{T}))d \tau$

$d\tilde{T} = (\frac{\alpha_1}{\mu^2})(c (\frac{\mu}{\alpha_1}\tilde{T}) (1 - d (\frac{\mu}{\alpha_1}\tilde{T})) - (\frac{\mu^2}{\alpha_1}) \tilde{N} \tilde{T} - \beta_2 (\frac{r_1 \mu^2}{\alpha_1^2 \alpha_2}) \tilde{L} \tilde{T})d \tau$

$d\tilde{T} = ((\frac{c}{\mu})\tilde{T} (1 - d (\frac{\mu}{\alpha_1}\tilde{T})) - \tilde{N} \tilde{T} - (\frac{\beta_2 r_1}{\alpha_1 \alpha_2}) \tilde{L} \tilde{T})d \tau$

$\tilde{T}' = \frac{c}{\mu}\tilde{T} (1 - \frac{d\mu }{\alpha_1}\tilde{T}) - \tilde{N} \tilde{T} - \frac{\beta_2 r_1}{\alpha_1 \alpha_2} \tilde{L} \tilde{T}$

For equation 4, this gives us:

$dD = (-\frac{\log(2)}{h} D_0 2^{-\frac{t}{h}})dt$

$d(\frac{T(0) \mu}{r_2} \tilde{D}) = (-\frac{\log(2)}{\frac{1}{\mu} \tilde{h}} D_0 2^{-\frac{\frac{1}{\mu} \tau}{\frac{1}{\mu} \tilde{h}}})(\frac{1}{\mu} d \tau)$

$d \tilde{D} = (\frac{r_2}{T(0) \mu^2})(-\frac{\mu \log(2)}{\tilde{h}} D_0 2^{-\frac{\tau}{\tilde{h}}})d \tau$

$d \tilde{D} = (-\frac{r_2 \log(2)}{\mu T(0) \tilde{h}} D_0 2^{-\frac{\tau}{\tilde{h}}})d \tau$

$\tilde{D}' = -\frac{r_2 \log(2)}{\mu T(0) \tilde{h}} D_0 2^{-\frac{\tau}{\tilde{h}}}$

So the non-dimensionalized system is

$\tilde{N}' = \frac{a_1}{\mu} \tilde{N} (1 - \frac{b \mu}{\alpha_2}\tilde{N}) - \frac{a_2}{\mu} \tilde{N} - \tilde{N} \tilde{T}$

$\tilde{L}' = \tilde{N} \tilde{T} + \frac{T(0) \alpha_1 \alpha_2}{r_1 \mu} \tilde{D} - \tilde{L} - \frac{\beta_1}{\alpha_1} \tilde{L} \tilde{T}$

$\tilde{T}' = \frac{c}{\mu}\tilde{T} (1 - \frac{d\mu }{\alpha_1}\tilde{T}) - \tilde{N} \tilde{T} - \frac{\beta_2 r_1}{\alpha_1 \alpha_2} \tilde{L} \tilde{T}$

$\tilde{D}' = -\frac{r_2 \log(2)}{\mu T(0) \tilde{h}} D_0 2^{-\frac{\tau}{\tilde{h}}}$

To further reduce the number of parameters, we define the following,

$p_1 = \frac{a_1}{\mu}$

$p_2 = \frac{a_2}{\mu}$

$q = \frac{b \mu}{\alpha_2}$

$r = \frac{T(0) \alpha_1 \alpha_2}{r_1 \mu}$

$s = \frac{\beta_1}{\alpha_1}$

$u = \frac{c}{\mu}$

$v = \frac{du}{\alpha_1}$

$\delta = \frac{\beta_2 r_1}{\alpha_1 \alpha_2}$

$D_0^* = \frac{r_2}{\mu T(0)} D_0$

And substitute:

$\tilde{N}' = p_1 \tilde{N} (1 - q \tilde{N}) - p_2 \tilde{N} - \tilde{N} \tilde{T}$

$\tilde{L}' = \tilde{N} \tilde{T} + r \tilde{D} - \tilde{L} - s \tilde{L} \tilde{T}$

$\tilde{T}' = u \tilde{T} (1 - v \tilde{T}) - \tilde{N} \tilde{T} - \delta \tilde{L} \tilde{T}$

$\tilde{D}' = -\frac{\log(2)}{\tilde{h}} D_0^* 2^{-\frac{\tau}{\tilde{h}}}$

Rewriting (but *not* substituting) the unitless symbols ($\tilde{N}$, $\tilde{L}$, $\tilde{T}$, $\tilde{D}$, $\tau$, $\tilde{h}$) with the symbols for their counterparts ($N$, $L$, $T$, $D$, $t$, $h$) yields:

$N' = p_1 N (1 - q N) - p_2 N - N T$

$L' = N T + r D - L - s L T$

$T' = u T (1 - v T) - N T - \delta L T$

$D' = -\frac{\log(2)}{h} D_0^* 2^{-\frac{t}{h}}$

### Consistency Check
Now we double-check the units (or lack thereof) of this new system.

In [11]:
C*k_c*alpha_1/mu**2

alpha_1*k_c*C(t)/mu**2

In [14]:
tilde_syms = {
    N: Function(r'\tilde{N}')(t/day),
    L: Function(r'\tilde{L}')(t/day),
    T: Function(r'\tilde{T}')(t/day),
    D: Function(r'\tilde{D}')(t/day),
    I_2: Function(r'\tilde{I_2}')(t/day),
    C: Function(r'\tilde{C}')(t/day),
}

T_0 = T.subs(t/day, 0)

subs = dict()
subs[N] = mu/alpha_2*tilde_syms[N]
subs[Np] = subs[N].diff(t/day)*mu
subs[L] = r_1*mu/alpha_1/alpha_2*tilde_syms[L]
subs[Lp] = subs[L].diff(t/day)*mu
subs[T] = mu/alpha_1*tilde_syms[T]
subs[Tp] = subs[T].diff(t/day)*mu
#subs[r_2*D] = r_2*T_0*mu/r_2*tilde_syms[D]
#subs[Dp] = subs[r_2*D].diff(t/day)*mu/r_2
subs[t] = tau/mu
subs[h] = symbols(r'\tilde{h}', real=True, positive=True)/mu

delta = symbols(r'\delta', real=True, positive=True)

parameter_subs = dict()
parameter_subs[a_1/mu] = symbols('p_1', real=True, positive=True)
parameter_subs[a_2/mu] = symbols('p_2', real=True, positive=True)
parameter_subs[b*mu/alpha_2] = symbols('q', real=True, positive=True)
parameter_subs[T.subs(t/day,0)*alpha_1*alpha_2/r_1/mu] = symbols('r', real=True, positive=True)
parameter_subs[beta_1/alpha_1] = symbols('s', real=True, positive=True)
parameter_subs[c/mu] = symbols('u', real=True, positive=True)
parameter_subs[d*mu/alpha_1] = symbols('v', real=True, positive=True)
parameter_subs[beta_2*r_1/alpha_1/alpha_2] = delta
#parameter_subs[r_2/mu/T_0*D_0] = symbols('D_0^*', real=True, positive=True)

print('Substitutions:')

for val, sub in subs.items():
    display(Eq(val, sub))

for val, sub in parameter_subs.items():
    display(Eq(sub, val))

print('Resulting equations:')

for var, equation in system.items():
    eq = expand(equation).subs(parameter_subs | subs)
    #display(expand(equation))
    #display(eq)
    eq_diff = Eq(tilde_syms[var].diff(t/day),
                 expand(solve(eq, tilde_syms[var].diff(t/day))[0])
                ).subs(parameter_subs)
    display(eq_diff)
    
    rhs_units = units(eq_diff.rhs)
    if rhs_units.has(Quantity):
        print('Something is wrong! The RHS has units ', rhs_units)
    else:
        print('Equation is unitless')

old_subs = subs

Substitutions:


Eq(cell*N(t), cell*mu*\tilde{N}(t)/alpha_2)

Eq(cell*Derivative(N(t), t)/day, cell*mu**2*Derivative(\tilde{N}(t), t)/(day*alpha_2))

Eq(cell*L(t), cell*mu*r_1*\tilde{L}(t)/(alpha_1*alpha_2))

Eq(cell*Derivative(L(t), t)/day, cell*mu**2*r_1*Derivative(\tilde{L}(t), t)/(day*alpha_1*alpha_2))

...

Eq(Derivative(\tilde{L}(t), t), -s*\tilde{L}(t)*\tilde{T}(t) - \tilde{L}(t) + \tilde{N}(t)*\tilde{T}(t))

Equation is unitless


Eq(Derivative(\tilde{T}(t), t), -\delta*\tilde{L}(t)*\tilde{T}(t) - alpha_1*k_c*C(t)/mu**2 - u*v*\tilde{T}(t)**2 + u*\tilde{T}(t) - \tilde{N}(t)*\tilde{T}(t))

Equation is unitless


IndexError: list index out of range

## Stability Analysis

Now we can construct the model with the nondimensionalized equations and simplified parameters for the stability analysis.

In [15]:
create_symbol_globals(parameter_subs.values())
D_0_s = symbols('D_0^*', real=True, positive=True)
k_n = symbols('k_n', real=True, positive=True)
k_t = symbols('k_t', real=True, positive=True)
ht = symbols(r'\tilde{h}', real=True, positive=True)
tt = symbols(r'\tilde{t}', real=True, positive=True)
new_param_definitions = {param: definition
                         for definition, param
                         in parameter_subs.items()}
new_param_definitions[ht] = mu*h
new_param_definitions[tt] = mu*t

subs = dict()

N = Function('N')(tt)
Np = N.diff(tt)

L = Function('L')(tt)
Lp = L.diff(tt)

T = Function('T')(tt)
Tp = T.diff(tt)

D = Function('D')(tt)
Dp = D.diff(tt)

nd_system = {
    N: Eq(Np, p_1*N*(1-q*N) - p_2*N - N*T + k_n*D),
    L: Eq(Lp, N*T + r*D - L - s*L*T),
    T: Eq(Tp, u*T*(1-v*T) - N*T - delta*L*T - k_t*D),
    D: Eq(Dp, -log(2)/ht*D_0_s*2**(-tt/ht)),
}

for equation in nd_system.values():
    display(equation)

Eq(Derivative(N(\tilde{t}), \tilde{t}), k_n*D(\tilde{t}) + p_1*(-q*N(\tilde{t}) + 1)*N(\tilde{t}) - p_2*N(\tilde{t}) - N(\tilde{t})*T(\tilde{t}))

Eq(Derivative(L(\tilde{t}), \tilde{t}), r*D(\tilde{t}) - s*L(\tilde{t})*T(\tilde{t}) - L(\tilde{t}) + N(\tilde{t})*T(\tilde{t}))

Eq(Derivative(T(\tilde{t}), \tilde{t}), -\delta*L(\tilde{t})*T(\tilde{t}) - k_t*D(\tilde{t}) + u*(-v*T(\tilde{t}) + 1)*T(\tilde{t}) - N(\tilde{t})*T(\tilde{t}))

Eq(Derivative(D(\tilde{t}), \tilde{t}), -D_0^**log(2)/(2**(\tilde{t}/\tilde{h})*\tilde{h}))

We then use these new equations to calculate the Jacobian of the system:

In [16]:
jacobian = simplify(Matrix([[equation.rhs.subs(subs).diff(var)
                             for var in nd_system.keys()
                            ] for equation in nd_system.values()]))
jacobian

Matrix([
[-2*p_1*q*N(\tilde{t}) + p_1 - p_2 - T(\tilde{t}),                    0,                                                -N(\tilde{t}),  k_n],
[                                    T(\tilde{t}),  -s*T(\tilde{t}) - 1,                               -s*L(\tilde{t}) + N(\tilde{t}),    r],
[                                   -T(\tilde{t}), -\delta*T(\tilde{t}), -\delta*L(\tilde{t}) - 2*u*v*T(\tilde{t}) + u - N(\tilde{t}), -k_t],
[                                               0,                    0,                                                            0,    0]])

In [17]:
jacobian = simplify(Matrix([[equation.rhs.subs(subs).diff(var)
                             for var in nd_system.keys()
                            ] for equation in nd_system.values()]))
jacobian

Matrix([
[-2*p_1*q*N(\tilde{t}) + p_1 - p_2 - T(\tilde{t}),                    0,                                                -N(\tilde{t}),  k_n],
[                                    T(\tilde{t}),  -s*T(\tilde{t}) - 1,                               -s*L(\tilde{t}) + N(\tilde{t}),    r],
[                                   -T(\tilde{t}), -\delta*T(\tilde{t}), -\delta*L(\tilde{t}) - 2*u*v*T(\tilde{t}) + u - N(\tilde{t}), -k_t],
[                                               0,                    0,                                                            0,    0]])

And calculate critical points:

In [ ]:
D.subs({tt: t/day})

In [ ]:
subs[D.subs({tt: t/day})] = 0

In [ ]:
subs[D] = 0

In [ ]:
subs = dict()

In [ ]:
solutions = dict()

for ix, (sym, equation) in enumerate(nd_system.items()):
    equation = equation.subs({tt: t/day})
    sym = sym.subs({tt: t/day})
    display(equation.subs(subs))
    pmd(f'Solutions for ${sym}$:')
    sols = solve(equation.subs(subs).rhs, sym)
    solutions[sym] = sols
    for sol in sols:
        display(Eq(sym, sol))
    if ix == 2:
        break
    print()

In [ ]:
v

In [18]:
lam = symbols('lambda', real=True)

In [ ]:
lam

In [ ]:
(lam+2e5*p_1*q-p_1+p_2+1e6)*(lam+1e6*s+1)*(lam+20*delta+2e5*u*v+u-1e6)*(lam-0)

In [ ]:
solveset(_23, lam, domain=S.Reals)

In [22]:
jacobian.subs({N: (p_1-p_2-T)/(p_1*q), L: N*T/(s*T+1), T: (-delta*L+u-N)/u/v, D: 0}).subs({-delta*L+u-N: z}).subs({tt: t/day})

Matrix([
[-p_1 + p_2 + z/(u*v),               0,                                                                                          -(p_1 - p_2 - z/(u*v))/(p_1*q),  k_n],
[             z/(u*v),  -s*z/(u*v) - 1,                                  -s*z*(p_1 - p_2 - z/(u*v))/(p_1*q*u*v*(s*z/(u*v) + 1)) + (p_1 - p_2 - z/(u*v))/(p_1*q),    r],
[            -z/(u*v), -\delta*z/(u*v), 2*\delta*L(t) - \delta*z*(p_1 - p_2 - z/(u*v))/(p_1*q*u*v*(s*z/(u*v) + 1)) - u + 2*N(t) - (p_1 - p_2 - z/(u*v))/(p_1*q), -k_t],
[                   0,               0,                                                                                                                       0,    0]])

In [19]:
lam*ones(4,1)-jacobian.diagonal().T

Matrix([
[            lambda + 2*p_1*q*N(\tilde{t}) - p_1 + p_2 + T(\tilde{t})],
[                                         lambda + s*T(\tilde{t}) + 1],
[\delta*L(\tilde{t}) + lambda + 2*u*v*T(\tilde{t}) - u + N(\tilde{t})],
[                                                              lambda]])

In [ ]:
prod(_45.args[2]).subs({N: (p_1-p_2-T)/(p_1*q), L: N*T/(s*T+1), T: (-delta*L+u-N)/u/v, D: 0})

In [21]:
solveset(prod(_19.args[2]).subs({N: (p_1-p_2-T)/(p_1*q), L: N*T/(s*T+1), T: (-delta*L+u-N)/u/v, D: 0}),lam,S.Reals)

Intersection({0, (\delta**3*L(\tilde{t})**2 - 2*\delta**2*p_1*q*s*u*v*L(\tilde{t})**2 + \delta**2*p_1*u*v*L(\tilde{t}) - \delta**2*p_2*u*v*L(\tilde{t}) + \delta**2*s*L(\tilde{t})**2 - 2*\delta**2*u*L(\tilde{t}) + 2*\delta**2*L(\tilde{t})*N(\tilde{t}) + 3*\delta*p_1*q*s*u**2*v*L(\tilde{t}) - 4*\delta*p_1*q*s*u*v*L(\tilde{t})*N(\tilde{t}) + 2*\delta*p_1*q*u**2*v**2*L(\tilde{t}) + \delta*p_1*s*u*v*L(\tilde{t}) - \delta*p_1*u**2*v + \delta*p_1*u*v*N(\tilde{t}) - \delta*p_2*s*u*v*L(\tilde{t}) + \delta*p_2*u**2*v - \delta*p_2*u*v*N(\tilde{t}) - 2*\delta*s*u*L(\tilde{t}) + 2*\delta*s*L(\tilde{t})*N(\tilde{t}) + \delta*u**2 - \delta*u*v*L(\tilde{t}) - 2*\delta*u*N(\tilde{t}) + \delta*N(\tilde{t})**2 - p_1*q*s*u**3*v + 3*p_1*q*s*u**2*v*N(\tilde{t}) - 2*p_1*q*s*u*v*N(\tilde{t})**2 - p_1*q*u**3*v**2 + 2*p_1*q*u**2*v**2*N(\tilde{t}) - p_1*s*u**2*v + p_1*s*u*v*N(\tilde{t}) - p_1*u**2*v**2 + p_2*s*u**2*v - p_2*s*u*v*N(\tilde{t}) + p_2*u**2*v**2 + s*u**2 - 2*s*u*N(\tilde{t}) + s*N(\tilde{t})**2 + u**

In [25]:
solveset(prod(_19.args[2]).subs({N: (p_1-p_2-T)/(p_1*q), L: N*T/(s*T+1), T: 0, D: 0}),lam,S.Reals)

{-1, 0, -p_1 + p_2, u - (p_1 - p_2)/(p_1*q)}

In [27]:
Matrix(_.args[1].args).subs({tt: t/day})

Matrix(0, 0, [])

In [29]:
Matrix(_25.args).subs({tt: t/day})

Matrix([
[                     -1],
[                      0],
[             -p_1 + p_2],
[u - (p_1 - p_2)/(p_1*q)]])

In [30]:
factor(_29)

Matrix([
[                     -1],
[                      0],
[             -p_1 + p_2],
[u - (p_1 - p_2)/(p_1*q)]])

In [35]:
_23

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              0],
[                                                                                                                                           

In [38]:
_35[3]

(\delta**3*L(t)**2 - 2*\delta**2*p_1*q*s*u*v*L(t)**2 + \delta**2*p_1*u*v*L(t) - \delta**2*p_2*u*v*L(t) + \delta**2*s*L(t)**2 - 2*\delta**2*u*L(t) + 2*\delta**2*L(t)*N(t) + 3*\delta*p_1*q*s*u**2*v*L(t) - 4*\delta*p_1*q*s*u*v*L(t)*N(t) + 2*\delta*p_1*q*u**2*v**2*L(t) + \delta*p_1*s*u*v*L(t) - \delta*p_1*u**2*v + \delta*p_1*u*v*N(t) - \delta*p_2*s*u*v*L(t) + \delta*p_2*u**2*v - \delta*p_2*u*v*N(t) - 2*\delta*s*u*L(t) + 2*\delta*s*L(t)*N(t) + \delta*u**2 - \delta*u*v*L(t) - 2*\delta*u*N(t) + \delta*N(t)**2 - p_1*q*s*u**3*v + 3*p_1*q*s*u**2*v*N(t) - 2*p_1*q*s*u*v*N(t)**2 - p_1*q*u**3*v**2 + 2*p_1*q*u**2*v**2*N(t) - p_1*s*u**2*v + p_1*s*u*v*N(t) - p_1*u**2*v**2 + p_2*s*u**2*v - p_2*s*u*v*N(t) + p_2*u**2*v**2 + s*u**2 - 2*s*u*N(t) + s*N(t)**2 + u**2*v - u*v*N(t))/(-\delta*p_1*q*s*u*v*L(t) + p_1*q*s*u**2*v - p_1*q*s*u*v*N(t) + p_1*q*u**2*v**2)

In [24]:
expand(_23[3]*p_1*q*u*v)

\delta**3*p_1*q*u*v*L(t)**2/(-\delta*p_1*q*s*u*v*L(t) + p_1*q*s*u**2*v - p_1*q*s*u*v*N(t) + p_1*q*u**2*v**2) - 2*\delta**2*p_1**2*q**2*s*u**2*v**2*L(t)**2/(-\delta*p_1*q*s*u*v*L(t) + p_1*q*s*u**2*v - p_1*q*s*u*v*N(t) + p_1*q*u**2*v**2) + \delta**2*p_1**2*q*u**2*v**2*L(t)/(-\delta*p_1*q*s*u*v*L(t) + p_1*q*s*u**2*v - p_1*q*s*u*v*N(t) + p_1*q*u**2*v**2) - \delta**2*p_1*p_2*q*u**2*v**2*L(t)/(-\delta*p_1*q*s*u*v*L(t) + p_1*q*s*u**2*v - p_1*q*s*u*v*N(t) + p_1*q*u**2*v**2) + \delta**2*p_1*q*s*u*v*L(t)**2/(-\delta*p_1*q*s*u*v*L(t) + p_1*q*s*u**2*v - p_1*q*s*u*v*N(t) + p_1*q*u**2*v**2) - 2*\delta**2*p_1*q*u**2*v*L(t)/(-\delta*p_1*q*s*u*v*L(t) + p_1*q*s*u**2*v - p_1*q*s*u*v*N(t) + p_1*q*u**2*v**2) + 2*\delta**2*p_1*q*u*v*L(t)*N(t)/(-\delta*p_1*q*s*u*v*L(t) + p_1*q*s*u**2*v - p_1*q*s*u*v*N(t) + p_1*q*u**2*v**2) + 3*\delta*p_1**2*q**2*s*u**3*v**2*L(t)/(-\delta*p_1*q*s*u*v*L(t) + p_1*q*s*u**2*v - p_1*q*s*u*v*N(t) + p_1*q*u**2*v**2) - 4*\delta*p_1**2*q**2*s*u**2*v**2*L(t)*N(t)/(-\delta*p_1*q*s*u*v*L

In [ ]:
#run_jacobian_steps(jacobian, {N: (p_1-p_2-T)/(p_1*q), L: N*T/(s*T+1), T: (-delta*L+u-N)/u/v, D: 0})
run_jacobian_steps(jacobian, {N: 1e5, L: 2e1, T: 1e6, D: 0})

Starting with the trivial solution ($N_0=L_0=T_0=D_0=0$) as a sanity check, we have

In [ ]:
run_jacobian_steps(jacobian, {N: 0, L: 0, T: 0, D: 0})

Since $u$ has a positive sign, $-1$ has a negative sign, and $p_1-p_2$ could be positive or negative, we can conclude that $O(0,0,0,0)$ is a saddle point.

Next we try $N_0=\frac{1}{q}$, $L_0=T_0=D_0=0$, to compare with the Zhang paper:

In [ ]:
run_jacobian_steps(jacobian, {N: 1/q, L: 0, T: 0, D: 0, p_2: 0})

Thankfully, these are essentially the same results as Zhang. The eigenvalues $-p_1$ and $-1$ are negative, while $u-\frac{1}{q}$ could be negative or positive. Since $u$ and $q$ are positive, $u - \frac{1}{q}$ will only be negative when $u < \frac{1}{q}$ or when $q < \frac{1}{u}$. So this set of initial conditions is a saddle point when $q > \frac{1}{u}$, and a stable sink when $0 < q < \frac{1}{u}$.

Next we check the critical point of $N$ with $N_0=\frac{p_1-p_2}{p_1 q}$, $L_0=T_0=D_0=0$.

In [ ]:
run_jacobian_steps(jacobian, {N: (p_1-p_2)/p_1/q, L: 0, T: 0, D: 0})

In [ ]:
run_jacobian_steps(jacobian, {N: (p_1-p_2+sqrt(4*k_n*p_1*q*D+p_1**2-2*p_1*p_2+p_2**2))/p_1/q/2, L: 0, T: 0, D: 0})

In [ ]:
(p_1-p_2+factor(sqrt(p_1**2+2*p_1*p_2+p_2**2)))/2/p_1/q

In [ ]:
run_jacobian_steps(jacobian, {N: 1/q, L: 0, T: 0, D: 0})

The eigenvalue $-1$ is negative, while $u+\frac{p_2}{p_1 q}-\frac{1}{q}$ and $p_2-p_1$ could be negative or positive. This is a fair bit more complicated, so we'll need a table from here on out.

| parameters | range | eigenvalue | stability |
|-----------|-------|------------|--------|
| $p_1$ | $(0, p_2)$ | $p_2-p_1 \gt 0$ | depends on $q$, or unstable source because $p_2 \in (p_1, \infty)$ |
| $p_1$ | $(p_2, \infty)$ | $p_2-p_1 \lt 0$ | saddle point |
| $p_2$ | $(p_1, \infty)$ | $u+\frac{1}{q}(\frac{p_2}{p1}-1) \gt 0$ | unstable source |
| $q$, $p_2$ | $q \in (0, \frac{1}{u}(1-\frac{p_2}{p_q}))$ and $p_2 \in (0, p_1)$ | $u+\frac{1}{q}(\frac{p_2}{p1}-1)$ | still thinking on this one |

## Drug Dosing Interval

In [ ]:
import scipy.interpolate

In [ ]:
concentrations = modeling_utils.mk_drug_concentration_table(11.23, 6*21)

In [ ]:
il2 = scipy.interpolate.CubicSpline(concentrations.index, concentrations['Interleukin-2'])
cyclo = scipy.interpolate.CubicSpline(concentrations.index, concentrations['Cyclophosphamide'])

In [ ]:
cyclo = scipy.interpolate.CubicSpline(
    concentrations.index,
    np.nan_to_num(concentrations['Cyclophosphamide'].shift(10*24*60))
)

In [ ]:
il2.derivative(

In [ ]:
model(0,[1,1,1,0,0])

In [ ]:
model_def = tomlkit.loads(Path('model.toml').read_text())
case = tomlkit.loads(Path('pillis-2008-no_tumor.toml').read_text())
base_params = tomlkit.loads(Path('parameters-mk03.toml').read_text())


display(modeling_utils.params2df(base_params['parameters']))


for risk_level, ivs in base_params['initial_values'].items():
    subs = ( modeling_utils.mk_subs_from_params(base_params['parameters'])
           | modeling_utils.mk_subs_from_params(ivs)
           #| {symbols('I_2'): il2, symbols('C'): cyclo}
           )
    
    display(modeling_utils.ivs2df(ivs))
    
    new_system = dict()
    
    for k, v in model_def['basic'].items():
        equation = sympify(v, {c: Function(c) for c in ivs.keys()})
        new_system[Function(k)(symbols('t'))] = Eq(symbols(k), equation)
        display(Eq(Function(k)(symbols('t')), equation.subs(subs)))
        #sympy.printing.tree.print_tree(equation, assumptions=False)
    
    model = mk_model(
        new_system,
        [symbols('t')] + [Function(c)(symbols('t')) for c in ivs.keys()],
        subs,
        [il2, cyclo],
    )
    
    #print([ivs['N'], ivs['L'], ivs['T'], ivs['D']])
    end_step = 21*3
    print(end_step)
    sol = solve_ivp(model, [0, end_step], list(ivs.values()), max_step=0.01)
    df_1 = pd.DataFrame(
        {'days': sol.t
        } | {sym: sol.y[ix]
                           for ix, sym in enumerate(ivs.keys())}
    ).set_index('days')
    df_1['I_2'] = il2(df_1.index)
    df_1['C'] = cyclo(df_1.index)
    ax = sns.lineplot(df_1)
    display(ax)
    ax = None
    
    fig, axes = plt.subplots(1, 5, figsize=(10,2))
    for ix, ax in enumerate(axes):
        sns.lineplot(df_1[df_1.columns[ix]], ax=ax)
        ax.set_title(df_1.columns[ix])
        if ix == 0:
            ax.yaxis.set_label_text('cells')
        else:
            ax.yaxis.label.set_visible(False)
    display(fig)

In [ ]:
cyclo = lambda t: 0

In [ ]:
import matplotlib.gridspec

In [ ]:
0.75/8

In [ ]:
[j**k for j in range(3) for k in range(3)]

In [ ]:

display(modeling_utils.params2df(base_params['parameters']).style.format('{:.4g}', subset='Value'))

In [ ]:
3.42e-3*1.0e-7

In [ ]:
df_1

In [ ]:
fig = make_subplots(
    rows=2, cols=6,
    specs=[[{"colspan": 2}, None, {"colspan": 2}, None, {"colspan": 2}, None],
           [{"colspan": 3}, None, None, {"colspan": 3}, None, None]],
    subplot_titles=['N', 'L', 'T', 'IL-2', 'Cyclophosphamide'],
    row_titles=['Cells', 'mg'],
    horizontal_spacing=0.05,
    vertical_spacing=0.125,
    print_grid=True,
)

In [ ]:
fig.add_trace(go.Scatter(x=df_1.index, y=df_1['N'], mode='lines'), row=1, col=1)
fig.add_trace(go.Scatter(x=df_1.index, y=df_1['L'], mode='lines'), row=1, col=3)
fig.add_trace(go.Scatter(x=df_1.index, y=df_1['T'], mode='lines'), row=1, col=5)
fig.add_trace(go.Scatter(x=df_1.index, y=df_1['I_2'], mode='lines'), row=2, col=1)
fig.add_trace(go.Scatter(x=df_1.index, y=df_1['C'], mode='lines'), row=2, col=4)
None

In [ ]:
Image(data=fig
      .update_layout(
          template='simple_white',
          margin_l=0,
          margin_r=0,
          margin_t=40,
          margin_b=0,
          showlegend=False,
      )
      .update_traces(row=1,col=1, line_color='#00aaff')
      .update_traces(row=1,col=3, line_color='green')
      .update_traces(row=1,col=5, line_color='orange')
      .update_traces(row=2,col=1, line_color='blue')
      .update_traces(row=2,col=4, line_color='purple')
      .update_yaxes(title_text="cells", row=1, col=1)
      .update_yaxes(title_text="mg", row=2, col=1)
      .update_traces(line_width=2.25)
      .to_image(format='png', scale=2))

In [ ]:
modeling_utils.mk_subplots(df_1).update_yaxes(exponentformat='power', row=1)

In [ ]:
Image(data=_.update_layout(height=450, width=750).to_image(format='png', scale=2))

In [ ]:
print(modeling_utils.ivs2df(ivs).style.format('{:.1e}', subset='Value').to_latex())

In [ ]:
model_def = tomlkit.loads(Path('model.txt').read_text())
base_params = tomlkit.loads(Path('parameters-mk04.txt').read_text())


display(modeling_utils.params2df(base_params['parameters']).style.format('{:.4g}', subset='Value'))

cases = list(base_params['initial_values'].items())

cases = cases[2:]

cases = [
    new_case
    for case in cases
    for new_case in [
        (*case,
         lambda t: 0,
         lambda t: 0),
    
        (*case,
         scipy.interpolate.CubicSpline(concentrations.index, concentrations['Interleukin-2']),
         lambda t: 0),
    
        (*case,
         lambda t: 0,
         scipy.interpolate.CubicSpline(concentrations.index, concentrations['Cyclophosphamide'])),
    
        (*case,
         scipy.interpolate.CubicSpline(concentrations.index, concentrations['Interleukin-2']),
         scipy.interpolate.CubicSpline(concentrations.index, concentrations['Cyclophosphamide'])),
    ]
    #(*cases[2],
    # scipy.interpolate.CubicSpline(concentrations.index, concentrations['Interleukin-2']),
    # scipy.interpolate.CubicSpline(
    #     concentrations.index,
    #     np.nan_to_num(concentrations['Cyclophosphamide'].shift(10*24*60)))),
]

for risk_level, ivs, il2, cyclo in cases:
    subs = ( modeling_utils.mk_subs_from_params(base_params['parameters'])
           | modeling_utils.mk_subs_from_params(ivs)
           #| {symbols('I_2'): il2, symbols('C'): cyclo}
           )
    
    display(modeling_utils.ivs2df(ivs).style.format('{:.1e}', subset='Value'))
    
    new_system = dict()
    
    for k, v in model_def['basic'].items():
        equation = sympify(v, {c: Function(c) for c in ivs.keys()})
        new_system[Function(k)(symbols('t'))] = Eq(symbols(k), equation)
        display(Eq(Function(k)(symbols('t')), equation.subs(subs)))
        #sympy.printing.tree.print_tree(equation, assumptions=False)
    
    model = mk_model(
        new_system,
        [symbols('t')] + [Function(c)(symbols('t')) for c in ivs.keys()],
        subs,
        [il2, cyclo],
        #[lambda t: 0, lambda t: 0],
    )
    
    #print([ivs['N'], ivs['L'], ivs['T'], ivs['D']])
    end_step = 21*6
    print(end_step)
    sol = solve_ivp(model, [0, end_step], list(ivs.values())
                    , max_step=1/60/24)
                    #, max_step=1)
    df_1 = pd.DataFrame(
        {'days': sol.t
        } | {sym: sol.y[ix]
                           for ix, sym in enumerate(ivs.keys())}
    ).set_index('days')
    df_1['I_2'] = il2(df_1.index)
    df_1['C'] = cyclo(df_1.index)
    ax = sns.lineplot(df_1)
    display(ax)
    ax = None
    
    #fig, axes = plt.subplots(1, 5, figsize=(10,2))
    figsize=(8,6)
    fig = plt.figure(figsize=figsize)
    gs = matplotlib.gridspec.GridSpec(2,6, wspace=figsize[0]/11, hspace=0.375)
    grid_specs = [
        ((0,0), 2),
        ((0,2), 2),
        ((0,4), 2),
        ((1,0), 3),
        ((1,3), 3),
    ]
    for ix, col in enumerate(df_1.columns):
        position, width = grid_specs[ix]
        ax = fig.add_subplot(gs.new_subplotspec(position, colspan=width))
        sns.lineplot(df_1[col], ax=ax)
        ax.set_title(col)
        if ix == 0:
            ax.yaxis.set_label_text('cells')
        elif ix == 3:
            ax.yaxis.set_label_text('mg')
        else:
            ax.yaxis.label.set_visible(False)
        #if ix in [2]:
        #    ax.set_yscale('log')
    display(fig)

In [ ]:
model_def = tomlkit.loads(Path('model.toml').read_text())
case = tomlkit.loads(Path('pillis-2008-no_tumor.toml').read_text())
base_params = tomlkit.loads(Path('parameters-mk04.toml').read_text())


display(modeling_utils.params2df(base_params['parameters']).style.format('{:.4g}', subset='Value'))

cases = list(base_params['initial_values'].items())

cases = cases[2:]

cases = [
    new_case
    for case in cases
    for new_case in [
        (*case,
         lambda t: 0,
         lambda t: 0),
    
        (*case,
         scipy.interpolate.CubicSpline(concentrations.index, concentrations['Interleukin-2']),
         lambda t: 0),
    
        (*case,
         lambda t: 0,
         scipy.interpolate.CubicSpline(concentrations.index, concentrations['Cyclophosphamide'])),
    
        (*case,
         scipy.interpolate.CubicSpline(concentrations.index, concentrations['Interleukin-2']),
         scipy.interpolate.CubicSpline(concentrations.index, concentrations['Cyclophosphamide'])),
    ]
    #(*cases[2],
    # scipy.interpolate.CubicSpline(concentrations.index, concentrations['Interleukin-2']),
    # scipy.interpolate.CubicSpline(
    #     concentrations.index,
    #     np.nan_to_num(concentrations['Cyclophosphamide'].shift(10*24*60)))),
]

for risk_level, ivs, il2, cyclo in cases:
    subs = ( modeling_utils.mk_subs_from_params(base_params['parameters'])
           | modeling_utils.mk_subs_from_params(ivs)
           #| {symbols('I_2'): il2, symbols('C'): cyclo}
           )
    
    display(modeling_utils.ivs2df(ivs).style.format('{:.1e}', subset='Value'))
    
    new_system = dict()
    
    for k, v in model_def['basic'].items():
        equation = sympify(v, {c: Function(c) for c in ivs.keys()})
        new_system[Function(k)(symbols('t'))] = Eq(symbols(k), equation)
        display(Eq(Function(k)(symbols('t')), equation.subs(subs)))
        #sympy.printing.tree.print_tree(equation, assumptions=False)
    
    model = mk_model(
        new_system,
        [symbols('t')] + [Function(c)(symbols('t')) for c in ivs.keys()],
        subs,
        [il2, cyclo],
        #[lambda t: 0, lambda t: 0],
    )
    
    #print([ivs['N'], ivs['L'], ivs['T'], ivs['D']])
    end_step = 21*6
    print(end_step)
    sol = solve_ivp(model, [0, end_step], list(ivs.values())
                    , max_step=1/60/24)
                    #, max_step=1)
    df_1 = pd.DataFrame(
        {'days': sol.t
        } | {sym: sol.y[ix]
                           for ix, sym in enumerate(ivs.keys())}
    ).set_index('days')
    df_1['I_2'] = il2(df_1.index)
    df_1['C'] = cyclo(df_1.index)
    ax = sns.lineplot(df_1)
    display(ax)
    ax = None
    
    #fig, axes = plt.subplots(1, 5, figsize=(10,2))
    figsize=(8,6)
    fig = plt.figure(figsize=figsize)
    gs = matplotlib.gridspec.GridSpec(2,6, wspace=figsize[0]/11, hspace=0.375)
    grid_specs = [
        ((0,0), 2),
        ((0,2), 2),
        ((0,4), 2),
        ((1,0), 3),
        ((1,3), 3),
    ]
    for ix, col in enumerate(df_1.columns):
        position, width = grid_specs[ix]
        ax = fig.add_subplot(gs.new_subplotspec(position, colspan=width))
        sns.lineplot(df_1[col], ax=ax)
        ax.set_title(col)
        if ix == 0:
            ax.yaxis.set_label_text('cells')
        elif ix == 3:
            ax.yaxis.set_label_text('mg')
        else:
            ax.yaxis.label.set_visible(False)
        #if ix in [2]:
        #    ax.set_yscale('log')
    display(fig)

In [ ]:
model_def = tomlkit.loads(Path('model.toml').read_text())
case = tomlkit.loads(Path('pillis-2008-no_tumor.toml').read_text())
base_params = tomlkit.loads(Path('parameters-mk03.toml').read_text())


display(modeling_utils.params2df(base_params['parameters']))


for risk_level, ivs in base_params['initial_values'].items():
    subs = ( modeling_utils.mk_subs_from_params(base_params['parameters'])
           | modeling_utils.mk_subs_from_params(ivs)
           #| {symbols('I_2'): il2, symbols('C'): cyclo}
           )
    
    display(modeling_utils.ivs2df(ivs))
    
    new_system = dict()
    
    for k, v in model_def['basic'].items():
        equation = sympify(v, {c: Function(c) for c in ivs.keys()})
        new_system[Function(k)(symbols('t'))] = Eq(symbols(k), equation)
        display(Eq(Function(k)(symbols('t')), equation.subs(subs)))
        #sympy.printing.tree.print_tree(equation, assumptions=False)
    
    model = mk_model(
        new_system,
        [symbols('t')] + [Function(c)(symbols('t')) for c in ivs.keys()],
        subs,
        #[il2, cyclo],
        [lambda t: 0, lambda t: 0],
    )
    
    #print([ivs['N'], ivs['L'], ivs['T'], ivs['D']])
    end_step = 21*3
    print(end_step)
    sol = solve_ivp(model, [0, end_step], list(ivs.values()), max_step=1/60/24)
    df_1 = pd.DataFrame(
        {'days': sol.t
        } | {sym: sol.y[ix]
                           for ix, sym in enumerate(ivs.keys())}
    ).set_index('days')
    df_1['I_2'] = il2(df_1.index)
    df_1['C'] = cyclo(df_1.index)
    ax = sns.lineplot(df_1)
    display(ax)
    ax = None
    
    fig, axes = plt.subplots(1, 5, figsize=(10,2))
    for ix, ax in enumerate(axes):
        sns.lineplot(df_1[df_1.columns[ix]], ax=ax)
        ax.set_title(df_1.columns[ix])
        if ix == 0:
            ax.yaxis.set_label_text('cells')
        else:
            ax.yaxis.label.set_visible(False)
    display(fig)

First we run a basic model with no tumor cells and a single dose of the drug.

In [ ]:
case = modeling_utils.import_case('pillis-2008-no_tumor.toml', globals())
case, params = case['case'], case['subs']
param_df = modeling_utils.case2df(case)
#params = modeling_utils.mk_subs_from_case(case, globals())
non_dim_params = {lhs: sympify(params.get(lhs, rhs.subs(params)))
                  for lhs, rhs in new_param_definitions.items() if lhs != tt }
non_dim_param_df = pd.DataFrame([{
    'Parameter': latex(sym, mode='inline'),
    'Definition': latex(new_param_definitions[sym], mode="equation"),
    'Value': val.n(4)}
 for sym, val in non_dim_params.items()]).set_index('Parameter').transpose()

display(param_df)
display(IPython.display.HTML(non_dim_param_df.to_html()))

model = mk_model(
    nd_system,
    [tt, N, L, T, D],
    {lhs: rhs.n() for lhs, rhs in non_dim_params.items()} | {k: v for k,v in params.items() if k != tt})

ivs = case['initial_values']
#print([ivs['N'], ivs['L'], ivs['T'], ivs['D']])
end_step = 20
print(end_step)
sol = solve_ivp(model, [0, end_step], [ivs['N'], ivs['L'], ivs['T'], ivs['D']])#, max_step=0.01)
df_1 = pd.DataFrame(
    {'scaled days': sol.t
    } | {sym.name: sol.y[ix]
                       for ix, sym in enumerate([N, L, T, D])}
).set_index('scaled days')
sns.lineplot(df_1)

In [ ]:
#case = modeling_utils.import_case('working-params.toml')
#param_df = modeling_utils.case2df(case)
case = modeling_utils.import_case('working-params.toml', globals())
case, params = case['case'], case['subs']
param_df = modeling_utils.case2df(case)
#params = modeling_utils.mk_subs_from_case(case, globals())
non_dim_params = {lhs: sympify(params.get(lhs, rhs.subs(params))) for lhs, rhs in new_param_definitions.items() if lhs != tt }
non_dim_param_df = pd.DataFrame([{
    'Parameter': latex(sym, mode='inline'),
    'Definition': latex(new_param_definitions[sym], mode="equation"),
    'Value': val.n(4)}
 for sym, val in non_dim_params.items()]).set_index('Parameter').transpose()

display(param_df)
display(IPython.display.HTML(non_dim_param_df.to_html()))

model = mk_model(
    nd_system,
    [tt, N, L, T, D],
    {lhs: rhs.n() for lhs, rhs in non_dim_params.items()} | {k: v for k,v in params.items() if k != tt})

ivs = case['initial_values']
#print([ivs['N'], ivs['L'], ivs['T'], ivs['D']])
end_step = 20
print(end_step)
sol = solve_ivp(model, [0, end_step], [ivs['N'], ivs['L'], ivs['T'], ivs['D']])#, max_step=0.01)
df_1 = pd.DataFrame(
    {'scaled days': sol.t
    } | {sym.name: sol.y[ix]
                       for ix, sym in enumerate([N, L, T, D])}
).set_index('scaled days')
sns.lineplot(df_1)

Everything looks good here! The NK cells are building up to their carrying capacity and CTL cells are declining after a brief boost from the drug. Now we can add in some tumor cells.

In [ ]:
model = mk_model(
    nd_system,
    [tt, N, L, T, D],
    {
        p_1: 1, q: 0.2, p_2: 0.5,
        r: 1, s: 1,
        u: 1, v: 0.1, delta: 0.1,
        ht: 1, D_0_s: 8,
    })

sol = solve_ivp(model, [0, 20], [2, 5, 1.5, 8], max_step=0.1)
df_2 = pd.DataFrame(
    {'scaled days': sol.t} | {sym.name: sol.y[ix]
                       for ix, sym in enumerate([N, L, T, D])}
).set_index('scaled days')
sns.lineplot(df_2)

In this simulation the tumor cells grow out of control, killing all the NK and CTL cells.

We can try adjusting this by administering additional doses of the drug when CTL count is below a certain level.

In [ ]:
model = mk_model(
    nd_system | {D: Eq(Dp,
                       Piecewise((5, L < 4),
                                 (nd_system[D].rhs, True)))},
    [tt, N, L, T, D],
    {
        p_1: 1, q: 0.2, p_2: 0.5,
        r: 1, s: 1,
        u: 1, v: 0.1, delta: 0.1,
        ht: 1, D_0_s: 8,
    })

sol = solve_ivp(model, [0, 20], [2, 5, 1.5, 8], max_step=0.1)
df_3 = pd.DataFrame(
    {'scaled days': sol.t} | {sym.name: sol.y[ix]
                       for ix, sym in enumerate([N, L, T, D])}
).set_index('scaled days')
sns.lineplot(df_3)

In this new simulation the tumor cells are suppressed thanks to the additional of doses of the CTL activating drug, and we can see NK cell count trending upwards towards the end of the graph.

In [ ]:
fig = px.line(
    pd.concat([df_1.assign(case=1),
               df_2.assign(case=2),
               df_3.assign(case=3)]),
    facet_col='case',
    width=1000,
    height=500
)
Image(data=fig.to_image(format='png'))

In [ ]:
import tomlkit

In [ ]:
tomlkit.loads(Path('case_0.toml').read_text())

In [ ]:
print(tomlkit.dumps({'parameters': {latex(sym, mode='inline'): {'Units': '', 'Description': '', 'Value': val, 'Source': ''} for sym, val in ({
        p_1: 1, q: 0.2, p_2: 0.5,
        r: 1, s: 1,
        u: 1, v: 0.1, delta: 0.1,
        ht: 1, D_0_s: 8,
    }).items()}}))

In [ ]:
tomlkit.loads(_24)

In [ ]:
_25['parameters']

In [ ]:
pd.DataFrame({f'${latex(symbols(sym))}$': vals for sym, vals in _27.items()}).transpose()

In [ ]:
import_params_from_clipboard_md()

In [ ]:
_119.transpose().to_dict()

In [ ]:
tomlkit.dumps({
    'initial_values': dict(N=0, L=1, T=2, D=3),
    'parameters': _121,
})

In [ ]:
Path('pillis-2008.toml').write_text(_177)

In [ ]:
pd.DataFrame(tomlkit.loads(Path('pillis-2008.toml').read_text())
            ).transpose().rename_axis(index='Parameter')

In [ ]:
def mk_subs_from_df(df):
    return {symbols(ix[1:-1].replace('\\',''),
                    real=True,
                    positive=True): row.Value
            for ix, row in _126.iterrows()}

In [ ]:
mk_subs_from_df(_130)

In [ ]:
IPython.display.HTML(pd.DataFrame([{'Parameter': latex(lhs, mode='inline'),
                                    'Definition': latex(rhs, mode="equation"),
                                  'Value': rhs.subs(_131 | {sympify('T(0)'): 2}).n(6)}
 for lhs, rhs in new_param_definitions.items()]).set_index('Parameter').transpose().to_html())

In [ ]:
for lhs, rhs in new_param_definitions.items():
    display(Eq(UnevaluatedExpr(Eq(lhs, rhs)), rhs.subs(_127)))

In [ ]:
for sym in [x[1:-1] for x in _74.index]:
    display(Eq(symbols(sym), parameter_subs.get(symbols(sym), -1)))

In [ ]:
pd.read_clipboard()

In [ ]:
pd.read_clipboard()

In [ ]:
_34.map(lambda x: x.strip() if type(x) == str else x)

In [ ]:
list(_69['Units '])

In [ ]:
pd.concat([
    (_34['Parameter ']
 .str.replace('₁', '_1')
 .str.replace('₂', '_2')
 .str.replace('α', '\\alpha')
 .str.replace('β', '\\beta')
 .str.replace('μ', '\\mu')
 .str.replace('I₀', 'D_0')
).apply(lambda x: f'${x.strip()}$'),
    _34[[c for c in _34.columns if 'Parameter' not in c]]], axis=1)

In [ ]:
print(_56.to_latex())

In [ ]:
tomlkit.dumps(_56.set_index('Parameter ').transpose().to_dict())

In [ ]:
new_param_definitions[symbols(list(_39.keys())[-2])]

In [ ]:
print(tomlkit.dumps(_27))

In [ ]:
model = mk_model(
    nd_system | {D: Eq(Dp,
                       Piecewise((5, Eq(sympy.floor(A) % 5, 0) & (T > 0.1)),
                                 (nd_system[D].rhs, True))),
                 A: Eq(A, 1)},
    [tt, N, L, T, D, A],
    {
        p_1: 1, q: 0.2, p_2: 0.5,
        r: 1, s: 1,
        u: 1, v: 0.1, delta: 0.1,
        ht: 1, D_0_s: 8,
    })

sol = solve_ivp(model, [0, 20], [2, 5, 1.5, 8, 0], max_step=0.05)
df_3 = pd.DataFrame(
    {'scaled days': sol.t} | {sym.name: sol.y[ix]
                       for ix, sym in enumerate([N, L, T, D])}
).set_index('scaled days')
px.line(df_3)

## Scratch

In [ ]:
kdl.ParseConfig(valueConverters=dict(
    mg=lambda a, val: float(a)))

In [ ]:
kdl.parse(Path('test.kdl').read_text(), config=_216)

In [ ]:
subs

In [ ]:
1/7

In [ ]:
list(new_system.values())[-1]

In [ ]:
t/day

In [ ]:
new_system

In [ ]:
integrate(list(new_system.values())[-2].rhs, t/day)

In [ ]:
list(_39.free_symbols)

In [ ]:
_39.args[1]

600,000 International Units/kg (0.037 mg/kg) IV q8hr infused over 15 minutes for a maximum 14 doses, THEN
9 days of rest, followed by a maximum of 14 more doses
Retreatment: Evaluate after 4 weeks, advisable only if tumor shrinkage and no retreatment contraindications (see package insert for details)

This medication is given by injection into a vein over 15 minutes by a health care professional. It may also be given in other ways as directed by your doctor.This medication is usually given every 8 hours for 5 days in a row. However, your doctor may decide to delay or stop your treatment depending on how you respond to this drug. After this treatment period, you will be given time to rest and recover before getting more of this medication. A course of treatment may include up to 28 doses of this medication. To make sure that you receive each scheduled dose as directed, it is important to keep all of your medical appointments while receiving this medication. Depending on your response, your doctor may decide that a second course would be helpful.Dosage is based on your medical condition, weight, response to treatment, and your side effects.

https://reference.medscape.com/drug/interleukin-2-proleukin-aldesleukin-342203#91

IL 2 was measured in the serum of patients, and a half-life of approximately 5 to 7 min was demonstrated with a second component of clearance of 30 to 120 min.

https://pubmed.ncbi.nlm.nih.gov/3871099/

In [ ]:
def mk_IL2_change(mass, halflife):
    dose = 0.037*mass #0.037 mg/kg
    last_dose = None
    max_levels = 0
    print(dose)

    def IL2_change(day, current_levels=dose):
        #nonlocal last_dose
        #nonlocal max_levels
        hour_of_day = day*24%24
        minute_of_hour = hour_of_day*60%60
        decay = 0

        #if last_dose is not None:
        #    decay = -dose*np.log(2)/(2**((day-last_dose)/halflife)*halflife)

        if 5 < day%14:
            decay += -dose*np.log(2)/(2**((day-last_dose)/halflife)*halflife)
            #return decay
        elif hour_of_day%8 > 1:
            decay += -dose*np.log(2)/(2**((minute_of_hour-(hour_of_day%8)*60-15)/halflife)*halflife)
            #return decay
        elif minute_of_hour > 15:
            decay += -dose*np.log(2)/(2**((minute_of_hour-15)/halflife)*halflife)
            #return decay
        else:
            #last_dose = day
            #max_levels = current_levels
            decay += dose/15

        if decay < 0 and current_levels <= 0:
            decay = 0

        return decay
    return IL2_change

In [ ]:
def mk_IL2_change(mass, halflife):
    dose = 0.037*mass #0.037 mg/kg
    last_dose = None
    max_levels = 0
    print(dose)

    def IL2_change(day, current_levels=0):
        nonlocal last_dose
        nonlocal max_levels
        hour_of_day = day*24%24
        minute_of_hour = hour_of_day*60%60
        decay = 0

        if last_dose is not None:
            decay = -max_levels*np.log(2)/(2**((day-last_dose)/halflife)*halflife)

        if 5 < day%14:
            return 'break'
        elif hour_of_day%8 > 1:
            return 'waiting for next hour'
        elif minute_of_hour > 15:
            return 'already dosed'
        else:
            return 'dosing'
    return IL2_change

In [ ]:
pd.DataFrame(dict(day=np.linspace(0,14,4*24*14))).assign(
    hour=lambda df: df['day']*24,
    concentration=lambda df: np.vectorize(mk_IL2_change(60,7))(df['day']),
).set_index('hour').drop(columns=['day'])

In [ ]:
px.line(_)

In [ ]:
pd.DataFrame(dict(day=np.linspace(0,14,60*24*14))).assign(
    hour=lambda df: df['day']*24,
    change=lambda df: np.vectorize(mk_IL2_change(60,7))(df['day']),
    concentration=lambda df: df['change'].cumsum(),
).set_index('hour').drop(columns=['day'])

In [ ]:
px.line(_.loc[:24])

In [ ]:
def mk_IL2_total(mass, halflife):
    dose = 0.037*mass #0.037 mg/kg
    last_dose = None
    max_levels = 0
    print(dose)

    def IL2_total(day):
        nonlocal last_dose
        nonlocal max_levels
        hour_of_day = day*24%24
        minute_of_hour = hour_of_day*60%60
        decay = 0
        levels = 0

        #if last_dose is not None:
        #    decay = -dose*np.log(2)/(2**((day-last_dose)/halflife)*halflife)

        if 5 < day%14:
            levels = max_levels*2**(-((day-last_dose)*60*24)/halflife)
            #decay += -dose*np.log(2)/(2**((day-last_dose)/halflife)*halflife)
            #return decay
        elif hour_of_day%8 > 1:
            levels = max_levels*2**(-((day-last_dose)*60*24)/halflife)
            #decay += -dose*np.log(2)/(2**((minute_of_hour-(hour_of_day%8)*60-15)/halflife)*halflife)
            #return decay
        elif minute_of_hour > 15:
            levels = max_levels*2**(-((day-last_dose)*60*24)/halflife)
            #decay += -dose*np.log(2)/(2**((minute_of_hour-15)/halflife)*halflife)
            #return decay
        else:
            last_dose = day
            levels = dose*minute_of_hour/15
            max_levels = levels

        return levels
    return IL2_total

In [ ]:
mk_IL2_total(60,7)

In [ ]:
pd.Series({t:_28(t) for t in np.linspace(0,21,21*24*60*5+1)})

In [ ]:
sns.lineplot(_)

In [ ]:
px.line(_)

In [ ]:
pd.DataFrame(dict(day=np.linspace(0,14,60*24*14))).assign(
    hour=lambda df: df['day']*24,
    change=lambda df: np.vectorize(mk_IL2_change(60,7))(df['day']),
    concentration=lambda df: df['change'].cumsum(),
).set_index('hour').drop(columns=['day'])

In [ ]:
mk_IL2_change(60,7/60/24)

In [ ]:
sol.y[0]

In [ ]:
import datetime

In [ ]:
sol = solve_ivp(
    mk_IL2_change(60,7),
    [0, 1],
    [0],
    #t_eval=np.linspace(0,12/24,10000),
    max_step=1/60/24,
    method='DOP853',
    rtol=1e-8,
    atol=1e-8)
df_1 = pd.DataFrame(
    {'days': sol.t,
     'concentration': sol.y[0]}
).set_index('days')
ax = sns.lineplot(df_1)
display(ax)
ax = None

In [ ]:
sol = solve_ivp(
    lambda t, y: 1,
    [1, 2],
    [1],
    #t_eval=np.linspace(0,12/24,10000),
    max_step=1/10,
    method='LSODA',
    rtol=1e-8,
    atol=1e-8)
df_1 = pd.DataFrame(
    {'days': sol.t,
     'concentration': sol.y[0]}
).set_index('days')
ax = sns.lineplot(df_1)
display(ax)
ax = None

In [ ]:
0.037*60

In [ ]:
integrate(1, (t,1,2))

In [ ]:
_39.subs({_43[0]: S(7)/60, _46: 0.037*60, _43[1]: Piecewise(
    (_43[1] % 8, _43[1]/24 < 5),
    (_43[1], True),)
})

In [ ]:
lambdify(_43[1], _120)

In [ ]:
pd.DataFrame(dict(day=np.linspace(0,14,100000))).assign(
    hour=lambda df: df['day']*24,
    concentration=lambda df: _121(df['hour']),
)#.set_index('day')

In [ ]:
sns.lineplot(_, y='concentration', x='day')

In [ ]:
integrate(list(new_system.values())[-1].rhs, t)

In [ ]:
[*list(_106.free_symbols), _106.args[1]]

IV: initial course, 40-50 mg/kg divided over 2-5 days; 10-15 mg/kg every 7-10 days; or 3-5 mg/kg twice weekly

https://reference.medscape.com/drug/cytoxan-cyclophosphamide-342214

In [ ]:
def mk_cyclo_total(mass, halflife):
    loading_dose = 40*mass # 40-50 mg/kg over 2-5 days
    loading_duration = 2
    recurring_dose = 10*mass # 10-15 mg/kg every 7-10 days
    dose_period = 7
    last_dose = None
    max_levels = 0
    had_recurring_dose = False

    def calc_levels(day):
        nonlocal last_dose
        nonlocal max_levels
        nonlocal had_recurring_dose

        hour_of_day = day*24%24
        minute_of_hour = hour_of_day*60%60
        decay = 0
        levels = 0

        if day <= loading_duration:
            levels = loading_dose/loading_duration
            max_levels = levels
            last_dose = day
            had_recurring_dose = True
        elif (day-loading_duration)%dose_period < 1 and not had_recurring_dose:
            levels = recurring_dose
            max_levels = levels
            last_dose = day
            had_recurring_dose = True
        else:
            levels = max_levels*2**(-(day-last_dose)/halflife)

            if (day-loading_duration)%dose_period > 1:
                had_recurring_dose = False

        return levels
    return calc_levels

In [ ]:
mk_cyclo_total(60,0.5)

In [ ]:
pd.Series({t:_47(t) for t in np.linspace(0,21,21*24*60+1)})

In [ ]:
px.line(_)

In [ ]:
12*60*60

In [ ]:
43200/60/60/24

In [ ]:
IL2_total = mk_IL2_total(81, 5)
cyclo_total = mk_cyclo_total(81, 537/60/24)

concentrations = pd.concat([
    pd.Series({t: cyclo_total(t) for t in np.linspace(0,21,21*24*60*5+1)}).rename('Cyclophosphamide'),
    pd.Series({t: IL2_total(t) for t in np.linspace(0,21,21*24*60*5+1)}).rename('Interleukin-2'),
], axis=1).rename_axis('Day')

In [ ]:
f'{7/60/24:e}'

In [ ]:
Image(data=px
      .line(concentrations.reset_index().melt(id_vars='Day', var_name='Drug', value_name='Concentration (mg)'),
            x='Day', y='Concentration (mg)', facet_col='Drug',
            height=600,
            width=1000,
            facet_col_spacing=0.04,
            title='Drug Concentration Over Time (assuming patient weighs 60kg)',
           )
      .update_yaxes(matches=None, showticklabels=True)
      .for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
      .to_image(format='png', scale=2))

In [ ]:
0.037*60

In [ ]:
_106.subs({_107[0]: 12, _107[2]: 5*60, _107[1]: _107[1] % 168})

In [ ]:
lambdify(_107[1], _109)

In [ ]:
pd.DataFrame(dict(day=np.linspace(0,14,10000))).assign(
    hour=lambda df: df['day']*24,
    concentration=lambda df: _110(df['hour']),
)#.set_index('day')

In [ ]:
sns.lineplot(_, y='concentration', x='day')

In [ ]:
model_def = tomlkit.loads(Path('model.toml').read_text())
case = tomlkit.loads(Path('pillis-2008-no_tumor.toml').read_text())
base_params = tomlkit.loads(Path('hybrid-parameters-mk01.toml').read_text())


display(modeling_utils.params2df(base_params['parameters']))


for risk_level, ivs in base_params['initial_values'].items():
    subs = ( modeling_utils.mk_subs_from_params(base_params['parameters'])
           | modeling_utils.mk_subs_from_params(ivs)
           )
    
    display(modeling_utils.ivs2df(ivs))
    
    new_system = dict()
    
    for k, v in model_def['basic'].items():
        equation = sympify(v, {c: Function(c) for c in ivs.keys()})
        new_system[Function(k)(symbols('t'))] = Eq(symbols(k), equation)
        #display(Eq(equation, equation.subs(subs)))
        #sympy.printing.tree.print_tree(equation, assumptions=False)
    
    model = mk_model(
        new_system,
        [symbols('t')] + [Function(c)(symbols('t')) for c in ivs.keys()],
        subs
    )
    
    #print([ivs['N'], ivs['L'], ivs['T'], ivs['D']])
    end_step = 300
    end_step = 400
    print(end_step)
    sol = solve_ivp(model, [0, end_step], list(ivs.values()), max_step=0.01)
    df_1 = pd.DataFrame(
        {'days': sol.t
        } | {sym: sol.y[ix]
                           for ix, sym in enumerate(ivs.keys())}
    ).set_index('days')
    ax = sns.lineplot(df_1)
    display(ax)
    ax = None
    
    fig, axes = plt.subplots(1, 3, figsize=(10,2))
    for ix, ax in enumerate(axes):
        sns.lineplot(df_1[df_1.columns[ix]], ax=ax)
        ax.set_title(df_1.columns[ix])
        if ix == 0:
            ax.yaxis.set_label_text('cells')
        else:
            ax.yaxis.label.set_visible(False)
    display(fig)

In [ ]:
model_def = tomlkit.loads(Path('model.toml').read_text())
case = tomlkit.loads(Path('pillis-2008-high_risk.toml').read_text())

base_params = tomlkit.loads(Path('pillis-2008.toml').read_text())

subs = ( modeling_utils.mk_subs_from_params(base_params['parameters'])
       | modeling_utils.mk_subs_from_params(case['initial_values'])
       )

#display(modeling_utils.ivs2df(case['initial_values']))
#display(modeling_utils.params2df(base_params['parameters']))

In [ ]:
pd.concat([
modeling_utils.ivs2df(base_params['initial_values']['low risk']).assign(Risk='Low'),
modeling_utils.ivs2df(base_params['initial_values']['intermediate risk']).assign(Risk='Intermediate'),
modeling_utils.ivs2df(base_params['initial_values']['high risk']).assign(Risk='High'),
]).reset_index()

In [ ]:
_147.melt(id_vars=['Parameter', 'Units', 'Description', 'Risk'])

In [ ]:
_169.pivot(index=['Parameter', 'Units', 'Description', 'Risk'], columns='variable', values='value').unstack().reset_index().set_index('Parameter')

In [ ]:
print(_173.to_latex())

In [ ]:
modeling_utils.params2df(base_params['parameters'])

In [ ]:
print(_131.to_latex(float_format='%.2e', bold_rows=True, ))

In [ ]:
new_system = dict()

for k, v in model_def['basic'].items():
    equation = sympify(v, {c: Function(c) for c in case['initial_values'].keys()})
    new_system[Function(k)(symbols('t'))] = Eq(symbols(k), equation)
    #display(Eq(equation, equation.subs(subs)))
    #sympy.printing.tree.print_tree(equation, assumptions=False)



model = mk_model(
    new_system,
    [symbols('t')] + [Function(c)(symbols('t')) for c in case['initial_values'].keys()],
    subs
)

ivs = case['initial_values']
#print([ivs['N'], ivs['L'], ivs['T'], ivs['D']])
end_step = 300
end_step = 400
print(end_step)
sol = solve_ivp(model, [0, end_step], list(case['initial_values'].values()), max_step=0.01)
df_1 = pd.DataFrame(
    {'days': sol.t
    } | {sym: sol.y[ix]
                       for ix, sym in enumerate(case['initial_values'].keys())}
).set_index('days')
sns.lineplot(df_1)

fig, axes = plt.subplots(1, 3, figsize=(10,2))
for ix, ax in enumerate(axes):
    sns.lineplot(df_1[df_1.columns[ix]], ax=ax)
    ax.set_title(df_1.columns[ix])
    if ix == 0:
        ax.yaxis.set_label_text('cells')
    else:
        ax.yaxis.label.set_visible(False)

In [ ]:

fig, axes = plt.subplots(1, 3, figsize=(10,2))
for ix, ax in enumerate(axes):
    sns.lineplot(df_1[df_1.columns[ix]], ax=ax)
    ax.set_title(df_1.columns[ix])
    if ix == 0:
        ax.yaxis.set_label_text('cells')
    else:
        ax.yaxis.label.set_visible(False)

In [ ]:
3/24

In [ ]:
fig

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(10,5))
for ix, ax in enumerate(axes):
    sns.lineplot(df_1[df_1.columns[ix]][90:], ax=ax)
    ax.set_title(df_1.columns[ix])

In [ ]:
df_1.diff()

In [ ]:
_112.rolling(1000).mean()

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(10,5))
for ix, ax in enumerate(axes):
    sns.lineplot(df_1.diff()[df_1.columns[ix]], ax=ax)
    ax.set_title(df_1.columns[ix])

In [ ]:
fig, axes = plt.subplots(1, 4)
for ix, ax in enumerate(axes):
    sns.lineplot(df_1[df_1.columns[ix]], ax=ax)

In [ ]:
sns.lineplot(df_1.drop(columns=['T']))

In [ ]:
sns.lineplot(df_1.drop(columns=['T', 'N']))

$N(t+1.5) = 2N(t)$

In [ ]:
t = symbols('t')

In [ ]:
1*2**(2*t/3)

In [ ]:
_29.diff(t)

In [ ]:
plot(
    2**(2*t/3),
    200/(1+exp(-0.4*(t-13))),
    #200/(1+exp(-0.6*(t-10.3))),
    #200/(1+exp(-0.8*(t-10.005))),
    (t, 0, 10))

In [ ]:
plot(
    2**(2*t/3),
    200/(1+exp(-0.4*(t-13))),
    200/(1+exp(-0.6*(t-10.3))),
    200/(1+exp(-0.8*(t-10.005))),
    (t, 0, 15))

In [ ]:
for val, sub in parameter_subs.items():
    print(sub,'=', val)